<div class="intro">
<br /><br />
    <center>
<span style="  font-size:2.75em;">
    <b> Feature Combination Work</b><br/>CGC<br /></span>
    </center>
        



# Combine Features

In [49]:
import pandas as pd
from pandas import DataFrame
from ast import literal_eval
import numpy as np

In [3]:
from datetime import date
today = str(date.today())

## Read in final features data

### CGC

In [5]:
cgc_final = pd.read_csv('/Volumes/Research/VulnerabilityDatasets/Vulnerability_datasets/code/Final Analysis Scripts/Jupyter/04_cgc_25Jan21.csv')

In [7]:
cgc_final.columns

Index(['function_count', 'external_funs', 'internal_funs', 'instructions',
       'fun_complexity', 'test_case_name', 'complexity_min', 'complexity_max',
       'complexity_mean', 'file_bytes', 'entropy', 'strings_set', 'CWE_ID',
       'CWE_pillar'],
      dtype='object')

### JULIET C

In [15]:
juliet_c_final = pd.read_csv('/Volumes/Research/VulnerabilityDatasets/Vulnerability_datasets/code/final_juliet_incl_pillars.csv')

In [16]:
juliet_c_final.columns

Index(['Unnamed: 0', 'path_x', 'function_count', 'external_funs',
       'internal_funs', 'instructions', 'fun_complexity', 'test_case_name',
       'complexity_min', 'complexity_max', 'complexity_mean', 'path_y',
       'file_bytes', 'entropy', 'strings_set', 'CWE', 'test_case_name_trimmed',
       'CWE_pillar'],
      dtype='object')

In [17]:
juliet_c_final = juliet_c_final.drop(columns=['Unnamed: 0', 'path_x', 'path_y', 'test_case_name_trimmed'])

In [18]:
juliet_c_final.columns

Index(['function_count', 'external_funs', 'internal_funs', 'instructions',
       'fun_complexity', 'test_case_name', 'complexity_min', 'complexity_max',
       'complexity_mean', 'file_bytes', 'entropy', 'strings_set', 'CWE',
       'CWE_pillar'],
      dtype='object')

In [19]:
juliet_c_final = juliet_c_final.rename(columns={"CWE": "CWE_ID"})

In [20]:
juliet_c_final.columns

Index(['function_count', 'external_funs', 'internal_funs', 'instructions',
       'fun_complexity', 'test_case_name', 'complexity_min', 'complexity_max',
       'complexity_mean', 'file_bytes', 'entropy', 'strings_set', 'CWE_ID',
       'CWE_pillar'],
      dtype='object')

#### Add CWE pillars to Juliet C

In [23]:
CWE_all = pd.read_csv('/Users/kaylakeen/PycharmProjects/datasetComparison/CWE_hierarchy_flattened.csv') # the CWE relationships database

In [24]:
def testcase_pillars(df):
   for index, row in df.iterrows(): # for each test_case in the list
      CWE_ID = str(df.at[index, "CWE_ID"])  # get the CWE data
      ids = [CWE_ID]

      #if this testcase_id has a cooresponding CWE_ID
      if ids[0] != [] and ids[0] != 'none':

         m = CWE_all.isin(ids).any() # check to see if the CWE_ID is in the dataframe
         cols = m.index[m].tolist() # if it is in the dataframe return a list of the columns that contain the value

         if cols != []: # as long as at least 1 match
            rows = CWE_all.index[CWE_all[cols[0]] == ids[0]].tolist() # use the first column to get the row
            pillar = CWE_all.at[rows[0], "Pillar_Level1"] # use the first row to get the pillar#
            df.at[index, 'CWE_pillar'] = pillar # update pillar info
   return(df) # return the updated df

In [25]:
final = testcase_pillars(juliet_c_final)

## Merge dfs

In [70]:
merged = final.append(cgc_final, ignore_index=True)

In [71]:
print(merged.columns)

Index(['function_count', 'external_funs', 'internal_funs', 'instructions',
       'fun_complexity', 'test_case_name', 'complexity_min', 'complexity_max',
       'complexity_mean', 'file_bytes', 'entropy', 'strings_set', 'CWE_ID',
       'CWE_pillar'],
      dtype='object')


In [72]:
print(merged[['CWE_ID','CWE_pillar']].isna())

       CWE_ID  CWE_pillar
0       False       False
1       False       False
2       False       False
3       False       False
4       False        True
...       ...         ...
42018   False       False
42019   False       False
42020   False       False
42021   False       False
42022   False       False

[42023 rows x 2 columns]


In [73]:
merged['CWE_pillar'].isnull().sum()

15285

In [74]:
merged[merged['CWE_pillar'].isnull()]

,function_count,external_funs,internal_funs,instructions,fun_complexity,test_case_name,complexity_min,complexity_max,complexity_mean,file_bytes,entropy,strings_set,CWE_ID,CWE_pillar
4,114.0,"[u'___stack_chk_fail', u'_pthread_exit']",['_CWE121_Stack_Based_Buffer_Overflow__CWE129_...,"['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...",[['_CWE121_Stack_Based_Buffer_Overflow__CWE129...,CWE121_Stack_Based_Buffer_Overflow__CWE129_con...,0.0,8.0,1.290909,21752,3.431892,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'1...",CWE121,NaN
14,112.0,"[u'___stack_chk_fail', u'_pthread_exit']",['_CWE121_Stack_Based_Buffer_Overflow__CWE129_...,"['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...",[['_CWE121_Stack_Based_Buffer_Overflow__CWE129...,CWE121_Stack_Based_Buffer_Overflow__CWE129_con...,0.0,8.0,1.148148,21208,3.169296,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'1...",CWE121,NaN
15,115.0,"[u'___stack_chk_fail', u'_pthread_exit']",['_CWE121_Stack_Based_Buffer_Overflow__CWE129_...,"['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...",[['_CWE121_Stack_Based_Buffer_Overflow__CWE129...,CWE121_Stack_Based_Buffer_Overflow__CWE129_con...,0.0,8.0,1.144144,21536,3.262049,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'1...",CWE121,NaN
16,118.0,"[u'___stack_chk_fail', u'_pthread_exit']",['_CWE121_Stack_Based_Buffer_Overflow__CWE129_...,"['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...",[['_CWE121_Stack_Based_Buffer_Overflow__CWE129...,CWE121_Stack_Based_Buffer_Overflow__CWE129_con...,0.0,8.0,1.140351,21864,3.343890,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'1...",CWE121,NaN
17,121.0,"[u'___stack_chk_fail', u'_pthread_exit']",['_CWE121_Stack_Based_Buffer_Overflow__CWE129_...,"['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...",[['_CWE121_Stack_Based_Buffer_Overflow__CWE129...,CWE121_Stack_Based_Buffer_Overflow__CWE129_con...,0.0,8.0,1.136752,22208,3.418222,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'1...",CWE121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41815,273.0,"[u'__throw_length_error', u'___cxa_throw', u'_...","['bad', 'vector', 'insert', 'end', '__ZNSt3__1...","['PUSH', 'MOV', 'SUB', 'LEA', 'MOV', 'MOV', 'C...","[['bad', 1], ['vector', 1], ['insert', 7], ['e...",CWE843_Type_Confusion__short_72.out,0.0,7.0,1.054902,42792,4.969787,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'F...",CWE843,NaN
41816,245.0,"[u'___cxa_throw', u'___stack_chk_fail', u'_pth...","['bad', 'list', 'push_back', 'list', '~list', ...","['PUSH', 'MOV', 'SUB', 'LEA', 'MOV', 'MOV', 'C...","[['bad', 1], ['list', 1], ['push_back', 1], ['...",CWE843_Type_Confusion__short_73.out,0.0,4.0,0.973799,37408,4.839857,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'F...",CWE843,NaN
41817,438.0,"[u'___cxa_throw', u'___stack_chk_fail', u'_pth...","['bad', 'map', 'operator[]', 'map', '~map', '_...","['PUSH', 'MOV', 'SUB', 'LEA', 'MOV', 'MOV', 'C...","[['bad', 1], ['map', 1], ['operator[]', 2], ['...",CWE843_Type_Confusion__short_74.out,0.0,10.0,1.101896,75272,5.284222,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'F...",CWE843,NaN
41818,103.0,"[u'___stack_chk_fail', u'_pthread_exit']","['action', 'action', 'bad', 'CWE843_Type_Confu...","['PUSH', 'MOV', 'SUB', 'MOV', 'MOV', 'MOV', 'M...","[['action', 1], ['action', 1], ['bad', 1], ['C...",CWE843_Type_Confusion__short_81.out,0.0,4.0,0.949495,23228,3.426551,"[{b'Finished bad()', b'%02x', b'%d -- %d', b'N...",CWE843,NaN


### Update Juliet C CWE ID format, add CWE pillar

In [83]:
merged['CWE_ID'] = merged['CWE_ID'].apply(lambda x: "CWE_".join(x.split("CWE")) if '_' not in x else x)

In [84]:
merged['CWE_ID']

0        CWE_121
1        CWE_121
2        CWE_121
3        CWE_121
4        CWE_121
          ...   
42018     CWE_20
42019     CWE_20
42020     CWE_20
42021     CWE_20
42022     CWE_20
Name: CWE_ID, Length: 42023, dtype: object

In [85]:
updated_merged = testcase_pillars(merged)

In [86]:
updated_merged['CWE_pillar'].isnull().sum()

0

### Send final to csv

In [88]:
updated_merged.to_csv('05_JULIETC_CGC_26Jan21.csv')